In [3]:
pip install opencv-python

SyntaxError: invalid syntax (<ipython-input-3-8a4382963592>, line 1)

In [8]:
# Import libraries
from collections import defaultdict
import cv2
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from random import choice, sample
from keras.applications.vgg16 import preprocess_input

In [9]:
# File paths
train_file_path = "../data/train/"
train_relationships_path = "../csv_files/train_relationships.csv"
validation_path = "F09"

In [10]:
# Functions by Youness Mansar https://github.com/CVxTz/kinship_prediction
all_images = glob(train_file_path + "*/*/*.jpg")

train_images = [x for x in all_images if validation_path not in x]
val_images = [x for x in all_images if validation_path in x]

train_person_to_images_map = defaultdict(list)

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv(train_relationships_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if validation_path not in x[0]]
val = [x for x in relationships if validation_path in x[0]]


def read_img(path):
    img = preprocessing.image.load_img(path, target_size=(160, 160))
    img = preprocessing.image.img_to_array(img)
#     img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels

In [12]:
img = read_img('../data/train/F0009/MID1/P10569_face2.jpg')

In [13]:
img.shape

(160, 160, 3)

In [14]:
# Import FaceNet model and FaceNet weights
facenet_model = models.load_model("../facenet/facenet_keras.h5")
facenet_model.load_weights("../facenet/facenet_keras_weights.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
# Model Summary 
facenet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
__________________________________________________________________________________________________
Conv2d_2a_

In [16]:
# Define Model Architecture 
def siamese_model():
    left_image = layers.Input(shape = (160, 160, 3))
    right_image = layers.Input(shape = (160, 160, 3))
    
    model = models.Sequential()
    model.add(facenet_model)
    model.add(layers.Dense(128, activation = "relu"))
    facenet_model.trainable = False
    
    x1 = model(left_image)
    x2 = model(right_image)
    
    L2_normalized_layer_1 = layers.Lambda(lambda x: K.l2_normalize(x, axis = 1))
    X1_normal = L2_normalized_layer_1(x1)
    X2_normal = L2_normalized_layer_1(x2)

    L1_layer = layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([X1_normal, X2_normal])
    
    prediction = layers.Dense(1, activation = "sigmoid")(L1_distance)
    
    siamese_net = models.Model(inputs = [left_image, right_image], outputs = prediction)

    siamese_net.compile(loss = "binary_crossentropy", metrics = ["acc"], optimizer = optimizers.Adam(0.00001))
    
    return siamese_net

In [17]:
kinship_model = siamese_model()
kinship_model.fit_generator(gen(train, train_person_to_images_map, batch_size = 16),
                    validation_data = gen(val, val_person_to_images_map, batch_size = 16), epochs = 100, verbose = 2,
                steps_per_epoch = 200, validation_steps = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
100/100 [==============================] - 7s 73ms/step - loss: 0.6975 - acc: 0.5031
 - 24s - loss: 0.7016 - acc: 0.5022 - val_loss: 0.6975 - val_acc: 0.5031
Epoch 2/100
100/100 [==============================] - 6s 62ms/step - loss: 0.6984 - acc: 0.4981
 - 19s - loss: 0.7006 - acc: 0.5059 - val_loss: 0.6984 - val_acc: 0.4981
Epoch 3/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6981 - acc: 0.5000
 - 19s - loss: 0.7016 - acc: 0.4969 - val_loss: 0.6981 - val_acc: 0.5000
Epoch 4/100
100/100 [==============================] - 6s 62ms/step - loss: 0.6956 - acc: 0.5081
 - 19s - loss: 0.6989 - acc: 0.4981 - val_loss: 0.6956 - val_acc: 0.5081
Epoch 5/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6985 - acc: 0.5000
 - 19s - loss: 0.6986 - acc: 0.5044 - val_loss: 0.6985 - val_acc: 0.5000
Epoch 6/100
100/100 [==============================] - 6s 62ms/step - loss: 0.6968 - acc: 0.5069
 - 19

100/100 [==============================] - 6s 61ms/step - loss: 0.6899 - acc: 0.5444
 - 19s - loss: 0.6910 - acc: 0.5375 - val_loss: 0.6899 - val_acc: 0.5444
Epoch 48/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6897 - acc: 0.5500
 - 19s - loss: 0.6914 - acc: 0.5297 - val_loss: 0.6897 - val_acc: 0.5500
Epoch 49/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6882 - acc: 0.5425
 - 19s - loss: 0.6897 - acc: 0.5322 - val_loss: 0.6882 - val_acc: 0.5425
Epoch 50/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6904 - acc: 0.5281
 - 19s - loss: 0.6910 - acc: 0.5272 - val_loss: 0.6904 - val_acc: 0.5281
Epoch 51/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6878 - acc: 0.5406
 - 19s - loss: 0.6915 - acc: 0.5284 - val_loss: 0.6878 - val_acc: 0.5406
Epoch 52/100
100/100 [==============================] - 6s 60ms/step - loss: 0.6872 - acc: 0.5475
 - 19s - loss: 0.6896 - acc: 0.5416 - val_loss: 0.6872 - val

Epoch 95/100
100/100 [==============================] - 6s 62ms/step - loss: 0.6851 - acc: 0.5656
 - 19s - loss: 0.6836 - acc: 0.5691 - val_loss: 0.6851 - val_acc: 0.5656
Epoch 96/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6856 - acc: 0.5594
 - 19s - loss: 0.6887 - acc: 0.5337 - val_loss: 0.6856 - val_acc: 0.5594
Epoch 97/100
100/100 [==============================] - 6s 62ms/step - loss: 0.6857 - acc: 0.5450
 - 19s - loss: 0.6851 - acc: 0.5622 - val_loss: 0.6857 - val_acc: 0.5450
Epoch 98/100
100/100 [==============================] - 6s 62ms/step - loss: 0.6868 - acc: 0.5525
 - 19s - loss: 0.6853 - acc: 0.5556 - val_loss: 0.6868 - val_acc: 0.5525
Epoch 99/100
100/100 [==============================] - 6s 62ms/step - loss: 0.6890 - acc: 0.5512
 - 19s - loss: 0.6838 - acc: 0.5691 - val_loss: 0.6890 - val_acc: 0.5512
Epoch 100/100
100/100 [==============================] - 6s 61ms/step - loss: 0.6848 - acc: 0.5725
 - 19s - loss: 0.6866 - acc: 0.5566 - val_loss

In [18]:
kinship_model_json = kinship_model.to_json()
with open("kinship_model.json", "w") as json_file:
    json_file.write(kinship_model_json)
    
kinship_model.save_weights("kinship_model_weights.h5")